In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LSTM
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
X.shape

(1142, 6250, 19)

In [8]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [9]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [10]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [11]:
run_experiment(X,y)

Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7184 - accuracy: 0.5235
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7086 - accuracy: 0.5071
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7143 - accuracy: 0.5126
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7175 - accuracy: 0.4962
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6968 - accuracy: 0.5225
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7185 - accuracy: 0.5181
Epoch 7/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6987 - accuracy: 0.5225
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7023 - accuracy: 0.5214
Epoch 9/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7012 - accuracy: 0.5148
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6870 - accuracy: 0.5455
Epoch 11/50
8/8 [============

8/8 [==============================] - 0s 49ms/step - loss: 0.6974 - accuracy: 0.5279
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6914 - accuracy: 0.5367
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6952 - accuracy: 0.5246
Epoch 37/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6956 - accuracy: 0.5159
Epoch 38/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6931 - accuracy: 0.5148
Epoch 39/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6972 - accuracy: 0.5257
Epoch 40/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6930 - accuracy: 0.5257
Epoch 41/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6949 - accuracy: 0.4940
Epoch 42/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6961 - accuracy: 0.4962
Epoch 43/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6948 - accuracy: 0.5181
Epoch 44/50
8/8 [===============

8/8 [==============================] - 0s 48ms/step - loss: 0.7071 - accuracy: 0.4940
Epoch 18/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7039 - accuracy: 0.4819
Epoch 19/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7099 - accuracy: 0.5027
Epoch 20/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6963 - accuracy: 0.5192
Epoch 21/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6811 - accuracy: 0.5652
Epoch 22/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6920 - accuracy: 0.5597
Epoch 23/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6985 - accuracy: 0.5411
Epoch 24/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6821 - accuracy: 0.5619
Epoch 25/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6988 - accuracy: 0.5411
Epoch 26/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6851 - accuracy: 0.5378
Epoch 27/50
8/8 [===============

8/8 [==============================] - 0s 49ms/step - loss: 0.6944 - accuracy: 0.5422
 >#5: 61.572 
[94.3231463432312, 61.57205104827881, 96.0698664188385, 96.50654792785645, 61.57205104827881]
 Accuracy: 82.009% (+/-16.702) 


In [12]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3),input_shape=(None,n_length,n_features)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3)))
    model.add(LeakyReLU())
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_2(X,y)

Epoch 1/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7377 - accuracy: 0.5115
Epoch 2/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7035 - accuracy: 0.5235
Epoch 3/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6922 - accuracy: 0.5356
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6754 - accuracy: 0.5783
Epoch 5/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6631 - accuracy: 0.5805
Epoch 6/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6629 - accuracy: 0.5958
Epoch 7/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6485 - accuracy: 0.6090
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6296 - accuracy: 0.6451
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5493 - accuracy: 0.7251
Epoch 10/50
8/8 [==============================] - 0s 50ms/step - loss: 0.4769 - accuracy: 0.7974
Epoch 11/50
8/8 [============

Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1855 - accuracy: 0.9354
Epoch 28/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1521 - accuracy: 0.9398
Epoch 29/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1578 - accuracy: 0.9496
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1350 - accuracy: 0.9452
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1280 - accuracy: 0.9562
Epoch 32/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1013 - accuracy: 0.9704
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1017 - accuracy: 0.9650
Epoch 34/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1036 - accuracy: 0.9639
Epoch 35/50
8/8 [==============================] - 0s 49ms/step - loss: 0.0843 - accuracy: 0.9726
Epoch 36/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0798 - accuracy: 0.9748
Epoch 37/50
8/8 [===

 >#3: 68.996 
Epoch 1/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7429 - accuracy: 0.5192
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7303 - accuracy: 0.5016
Epoch 3/50
8/8 [==============================] - 0s 48ms/step - loss: 0.7187 - accuracy: 0.5071
Epoch 4/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7127 - accuracy: 0.5181
Epoch 5/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7021 - accuracy: 0.5323
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6836 - accuracy: 0.5509
Epoch 7/50
8/8 [==============================] - 0s 48ms/step - loss: 0.6558 - accuracy: 0.6024
Epoch 8/50
8/8 [==============================] - 0s 49ms/step - loss: 0.6170 - accuracy: 0.6484
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5993 - accuracy: 0.6791
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 0.5351 - accuracy: 0.7382
Epoch 11/50
8/8

8/8 [==============================] - 0s 49ms/step - loss: 0.3590 - accuracy: 0.8729
Epoch 27/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3243 - accuracy: 0.8719
Epoch 28/50
8/8 [==============================] - 0s 50ms/step - loss: 0.3375 - accuracy: 0.8697
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2522 - accuracy: 0.9266
Epoch 30/50
8/8 [==============================] - 0s 50ms/step - loss: 0.2564 - accuracy: 0.9058
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1798 - accuracy: 0.9365
Epoch 32/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2146 - accuracy: 0.9266
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 0.2142 - accuracy: 0.9321
Epoch 34/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1831 - accuracy: 0.9398
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1474 - accuracy: 0.9518
Epoch 36/50
8/8 [===============

In [13]:
def evaluate_model_3(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 50, 128
    n_features = trainX.shape[2]
    n_steps, n_length = 10, 625
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu'),
    input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='selu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

def run_experiment_3(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_3(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    
run_experiment_3(X,y)

Epoch 1/50
8/8 [==============================] - 0s 53ms/step - loss: 0.7981 - accuracy: 0.4984
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7350 - accuracy: 0.5027
Epoch 3/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7297 - accuracy: 0.5115
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7224 - accuracy: 0.5455
Epoch 5/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7100 - accuracy: 0.5312
Epoch 6/50
8/8 [==============================] - 0s 50ms/step - loss: 0.7006 - accuracy: 0.5652
Epoch 7/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6955 - accuracy: 0.5608
Epoch 8/50
8/8 [==============================] - 0s 52ms/step - loss: 0.6644 - accuracy: 0.6112
Epoch 9/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6390 - accuracy: 0.6188
Epoch 10/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6109 - accuracy: 0.6824
Epoch 11/50
8/8 [============

Epoch 27/50
8/8 [==============================] - 0s 52ms/step - loss: 0.2519 - accuracy: 0.9146
Epoch 28/50
8/8 [==============================] - 0s 49ms/step - loss: 0.2097 - accuracy: 0.9277
Epoch 29/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2229 - accuracy: 0.9277
Epoch 30/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2092 - accuracy: 0.9277
Epoch 31/50
8/8 [==============================] - 0s 49ms/step - loss: 0.2204 - accuracy: 0.9244
Epoch 32/50
8/8 [==============================] - 0s 49ms/step - loss: 0.2164 - accuracy: 0.9233
Epoch 33/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2204 - accuracy: 0.9233
Epoch 34/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2216 - accuracy: 0.9255
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1755 - accuracy: 0.9376
Epoch 36/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1969 - accuracy: 0.9277
Epoch 37/50
8/8 [===

 >#3: 48.035 
Epoch 1/50
8/8 [==============================] - 0s 51ms/step - loss: 0.7780 - accuracy: 0.5192
Epoch 2/50
8/8 [==============================] - 0s 49ms/step - loss: 0.7152 - accuracy: 0.5772
Epoch 3/50
8/8 [==============================] - 0s 51ms/step - loss: 0.6932 - accuracy: 0.5805
Epoch 4/50
8/8 [==============================] - 0s 50ms/step - loss: 0.6422 - accuracy: 0.6232
Epoch 5/50
8/8 [==============================] - 0s 50ms/step - loss: 0.5661 - accuracy: 0.7207
Epoch 6/50
8/8 [==============================] - 0s 51ms/step - loss: 0.4367 - accuracy: 0.8182
Epoch 7/50
8/8 [==============================] - 0s 52ms/step - loss: 0.4019 - accuracy: 0.8324
Epoch 8/50
8/8 [==============================] - 0s 51ms/step - loss: 0.3098 - accuracy: 0.8729
Epoch 9/50
8/8 [==============================] - 0s 51ms/step - loss: 0.2585 - accuracy: 0.9058
Epoch 10/50
8/8 [==============================] - 0s 49ms/step - loss: 0.2421 - accuracy: 0.9058
Epoch 11/50
8/8

8/8 [==============================] - 0s 49ms/step - loss: 0.1730 - accuracy: 0.9376
Epoch 27/50
8/8 [==============================] - 0s 52ms/step - loss: 0.1370 - accuracy: 0.9518
Epoch 28/50
8/8 [==============================] - 0s 48ms/step - loss: 0.1679 - accuracy: 0.9441
Epoch 29/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1439 - accuracy: 0.9496
Epoch 30/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1471 - accuracy: 0.9452
Epoch 31/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1504 - accuracy: 0.9518
Epoch 32/50
8/8 [==============================] - 0s 50ms/step - loss: 0.0943 - accuracy: 0.9693
Epoch 33/50
8/8 [==============================] - 0s 50ms/step - loss: 0.1204 - accuracy: 0.9671
Epoch 34/50
8/8 [==============================] - 0s 49ms/step - loss: 0.1378 - accuracy: 0.9540
Epoch 35/50
8/8 [==============================] - 0s 51ms/step - loss: 0.1563 - accuracy: 0.9419
Epoch 36/50
8/8 [===============